In [2]:
import tensorflow as tf
import os

In [3]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

num_shards = 2
instances_per_shard = 2

for i in range(num_shards):
    filename = os.path.join("data", "tfrecords-%.5d-of-%.5d" % (i, num_shards))
    writer = tf.python_io.TFRecordWriter(filename)
    for j in range(instances_per_shard):
        example = tf.train.Example(features=tf.train.Features(feature={'i': _int64_feature(i),
                                                                      'j': _int64_feature(j)}))
        writer.write(example.SerializeToString())
    writer.close()

In [6]:
files = tf.train.match_filenames_once(os.path.join('data', 'tfrecords-*'))
filename_queue = tf.train.string_input_producer(files, shuffle=False)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                  features={
                                      'i': tf.FixedLenFeature([], tf.int64),
                                      'j': tf.FixedLenFeature([], tf.int64)
                                  })
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    print(sess.run(files))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
    coord.request_stop()
    coord.join(threads)

[b'data/tfrecords-00000-of-00002' b'data/tfrecords-00001-of-00002']
[0, 0]
[0, 1]
[1, 0]
[1, 1]
[0, 0]
[0, 1]


In [8]:
files = tf.train.match_filenames_once(os.path.join('data', 'tfrecords-*'))
filename_queue = tf.train.string_input_producer(files, shuffle=False)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                  features={
                                      'i': tf.FixedLenFeature([], tf.int64),
                                      'j': tf.FixedLenFeature([], tf.int64)
                                  })
example, label = features['i'], features['j']

batch_size = 3
capacity = 1000 + 3 * batch_size

example_batch, label_batch = tf.train.batch([example, label], batch_size=batch_size, capacity=capacity)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
    coord.request_stop()
    coord.join(threads)

[0 0 1] [0 1 0]
[1 0 0] [1 0 1]


In [10]:
files = tf.train.match_filenames_once(os.path.join('data', 'tfrecords-*'))
filename_queue = tf.train.string_input_producer(files, shuffle=False)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                  features={
                                      'i': tf.FixedLenFeature([], tf.int64),
                                      'j': tf.FixedLenFeature([], tf.int64)
                                  })
example, label = features['i'], features['j']

batch_size = 3
capacity = 1000 + 3 * batch_size

example_batch, label_batch = tf.train.shuffle_batch([example, label], batch_size=batch_size, capacity=capacity,
                                           min_after_dequeue=30)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
    coord.request_stop()
    coord.join(threads)

[1 0 1] [0 1 1]
[0 0 0] [0 0 1]
